Importing the necessary packages and models
---
Installing pyfluidsynth and fluidsynth:

In [ ]:
# Instalation / Setting up the environment.

%tensorflow_version 1.x

# Dependencies.
print("**************************************************************")
print('Installing dependencies...')

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("**************************************************************")
print("The specified dependencies have been successfully installed.")
print('🎉 Done!')

TensorFlow 1.x selected.
**************************************************************
Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6MB 6.4MB/s 
     |████████████████████████████████| 1.6MB 4.6MB/s 
     |████████████████████████████████| 204kB 25.6MB/s 
     |████████████████████████████████| 1.6MB 32.4MB/s 
 

Packages and libraries:

In [ ]:
# Libraries.
print("**************************************************************")
print('Importing libraries...')

from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

import cv2
import numpy as np
from sklearn.cluster import KMeans

from google.colab import files
from google.colab.patches import cv2_imshow

import magenta.music as mm
import magenta
import tensorflow
import ast
import os
import time
import pandas as pd

from magenta.models.performance_rnn import performance_model
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import constants
from magenta.music.protobuf import generator_pb2
from magenta.music.protobuf import music_pb2
from magenta.music import midi_synth

print("**************************************************************")
print("The specified libraries have been successfully imported.")
print("**************************************************************")
print("Magenta version " + magenta.__version__)
print("Tensorflow version "+ tensorflow.__version__)
print('🎉 Done!')

**************************************************************
Importing libraries...
**************************************************************
The specified libraries have been successfully imported.
**************************************************************
Magenta version 1.3.1
Tensorflow version 1.15.2
🎉 Done!


SoundFont:

In [ ]:
# Downloading and setting a SoundFont.

# Samples by Alexander Holm: https://archive.org/details/SalamanderGrandPianoV3
# Converted to sf2 by John Nebauer: https://sites.google.com/site/soundfonts4u
!gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/

print("**************************************************************")
print("SoundFont successfully downloaded.") 
print('🎉 Done!')

Copying gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2...
\ [1/1 files][591.9 MiB/591.9 MiB] 100% Done                                    
Operation completed over 1 objects/591.9 MiB.                                    
**************************************************************
SoundFont successfully downloaded.
🎉 Done!


Models:

In [ ]:
# Downloading and installing the model. 

# Constants.
#BUNDLE_DIR = '/tmp/'
MODEL_NAME = 'multiconditioned_performance_with_dynamics'
BUNDLE_NAME = MODEL_NAME + '.mag'

mm.notebook_utils.download_bundle(BUNDLE_NAME, "bundles")
bundle = sequence_generator_bundle.read_bundle_file(os.path.join("bundles", BUNDLE_NAME))

print("**************************************************************")
print("Bundle has been downloaded successfully")
print('🎉 Done!')

**************************************************************
Bundle has been downloaded successfully
🎉 Done!


Image recognition section:
---

Code to access camera from Google Colab provided by Google (with some minor modifications)

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      //const capture = document.createElement('button');
      //capture.textContent = 'Capture';
      //div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename
print("Method for taking photos successfully built")

Method for taking photos successfully built


Methods for processing the images

In [ ]:
# Brightness detection method.
def rgbBright(img):
    #newImg = np.asarray(img)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #hsv = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2HSV)
    return hsv[...,2].mean()

# Making a histogram of the colors by grouping them with k cluster.
def make_histogram(cluster):
    """
    Count the number of pixels in each cluster
    :param: KMeans cluster
    :return: numpy histogram
    """
    numLabels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
    hist, _ = np.histogram(cluster.labels_, bins=numLabels)
    hist = hist.astype('float32')
    hist /= hist.sum()
    return hist

def make_bar(height, width, color):
    """
    Create an image of a given color
    :param: height of the image
    :param: width of the image
    :param: BGR pixel values of the color
    :return: tuple of bar, rgb values, and hsv values
    """
    bar = np.zeros((height, width, 3), np.uint8)
    bar[:] = color
    red, green, blue = int(color[2]), int(color[1]), int(color[0])
    hsv_bar = cv2.cvtColor(bar, cv2.COLOR_BGR2HSV)
    hue, sat, val = hsv_bar[0][0]
    return bar, (red, green, blue), (hue, sat, val)

def sort_hsvs(hsv_list):
    """
    Sort the list of HSV values
    :param hsv_list: List of HSV tuples
    :return: List of indexes, sorted by hue, then saturation, then value
    """
    bars_with_indexes = []
    for index, hsv_val in enumerate(hsv_list):
        bars_with_indexes.append((index, hsv_val[0], hsv_val[1], hsv_val[2]))
    bars_with_indexes.sort(key=lambda elem: (elem[1], elem[2], elem[3]))
    return [item[0] for item in bars_with_indexes]

def photoAnalyse(img):
    height, width, _ = np.shape(img)

    # Print brightness.
    #print("Brightness")
    #print(str(rgbBright(img)))

    # Reshape the image to be a simple list of RGB pixels.
    image = img.reshape((height * width, 3))

    # We'll pick the 6 most common colors.
    num_clusters = 6
    clusters = KMeans(n_clusters=num_clusters)
    clusters.fit(image)

    # Count the dominant colors and put them in "buckets".
    histogram = make_histogram(clusters)
    # Then sort them, most-common first.
    combined = zip(histogram, clusters.cluster_centers_)
    combined = sorted(combined, key=lambda x: x[0], reverse=True)

    # Finally, we'll output a graphic showing the colors in order.
    bars = []
    hsv_values = []

    for index, rows in enumerate(combined):
        bar, rgb, hsv = make_bar(100, 100, rows[1])
        #print(f'Bar {index + 1}')
        #print(f'  RGB values: {rgb}')
        #print(f'  HSV values: {hsv}')
        hsv_values.append(hsv)
        bars.append(bar)
    """
    # Code to view the most common colors and the image taken
    # Sort the bars[] list so that we can show the colored boxes sorted
    # by their HSV values -- sort by hue, then saturation.
    sorted_bar_indexes = sort_hsvs(hsv_values)
    sorted_bars = [bars[idx] for idx in sorted_bar_indexes]
    
    cv2_imshow(np.hstack(sorted_bars))
    

    #"""
    return hsv_values
print("Image analysis methods successfully built")

Image analysis methods successfully built


Music generation section:
---

Rules and methods that define the musical piece:

In [ ]:
# Rules to consider in order to generate the melody.

# Auxiliary method that defines the frequency of notes associated with the red
# color.
def red_hue (v, pitches):
  if v <= 54:
    # Dark red = F#.
    pitches[6] = 1
  else:
    # Red = G and G#.
    pitches[7] = 1
    pitches[8] = 1

  return pitches

# Auxiliary method that defines the frequency of notes associated with the 
# yellow color.
def yellow_hue(hue,pitches):
  # Hue 67 = lime = C.
  if hue >= 67:
    pitches[0] = 1
  # Hue 60 = yellow = #A.
  else:
    pitches[10] = 1

  return pitches

# Auxiliary method that defines the frequency of notes associated with the
# blue color.
def blue_hue(hue,pitches):
  # Hue 247 = indigo = F.
  if hue >= 247:
    pitches[6] = 1
  # Hue 240 = blue = D and E.
  else:
    pitches[3] = 1
    pitches[4] = 1 

  return pitches

# pitch_class_histogram: A string representation of a Python list of 12 values 
#                        representing the relative frequency of notes of each 
#                        pitch class, starting with C.
def pitch_hist (hsv_values):
  h = 0
  v = 2
  # Representation of the chromatic scale.
  pitches = [0,0,0,0,0,0,0,0,0,0,0,0]

  place = range(5)
  for n in place:
    hue = hsv_values[n][0]
    # Evaluate which color corresponds to the hue.
    aux = hue // 10
    
    if aux < 3:
      pitches = red_hue(hsv_values[n][v],pitches)
    elif aux < 5:
      # Hue 30 = orange = A.
      pitches[9] = 1
    elif aux < 11:
      pitches = yellow_hue(hue,pitches)
    elif aux < 17:
      # Hue 120 = green = B.
      pitches[11] = 1
    elif aux <= 18:
      # Hue 180 = aqua = C#.
      pitches[1] = 1
    elif aux < 23:
      # Hue 194 = light blue = D.
      pitches[3] = 1
    else: 
      pitches = blue_hue(hue,pitches)

  return pitches

# notes_per_second: The desired number of notes per second in the output 
#                   performance. 
#                   Top value is set to 15 nps.
def notes_sec (brightness_value):
    # value is found within the first 3 categories.
    if brightness_value <= 80:
      if brightness_value <= 26:
        notes = 4.0
      elif brightness_value <= 53:
        notes = 5.0
      else:
        notes = 6.0
    # value is found in the last 3 categories.
    else:
      if brightness_value <= 107:
        notes = 7.0
      elif brightness_value <= 134:
        notes = 8.0
      else:
        notes = 9.0
    return notes

print("Music generator methods successfully built")

Music generator methods successfully built


Generating performance:

In [ ]:
# Generating performance.
def performance(nps, pitch, length, temp = 1.0):

      generator_map = performance_sequence_generator.get_generator_map()
      generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)
      generator.initialize()
      generator_options = generator_pb2.GeneratorOptions()

      # temperature: Greater than 1.0 makes tracks more random, less than 1.0 
      #              makes tracks less random.
      #              0.75 is default.
      generator_options.args['temperature'].float_value = temp 

      # notes per second using brightness level: Note that increasing this value 
      #                                          will cause generation to take
      #                                          longer, as the number of RNN 
      #                                          steps is roughly proportional
      #                                          to the number of  notes
      #                                          generated. 
      generator_options.args['notes_per_second'].string_value = str(nps) 

      # pitch class histogram using colors: For example
      #                                     [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
      #                                     [C, C#, D, D#, E, F, F#, G, G#, A, A#, B]
      #                                     will tend to stick to a C-major scale, 
      #                                     with twice as much C as any of the other 
      #                                     notes of the scale.
      generator_options.args['pitch_class_histogram'].string_value = str(pitch)

      # start time and end time: Set to 60 seconds by default to keep examples 
      #                          a manageable size.
      end = length

      generate_section = generator_options.generate_sections.add(start_time=0, end_time=end)

      # Generate sequence.
      sequence = generator.generate(music_pb2.NoteSequence(), generator_options)
      #Transform sequence to a 2d float array
      synth=mm.fluidsynth
      sequenceArr = synth(sequence, sample_rate=44100)
      #Play music piece
      mm.notebook_utils.colab_play(sequenceArr, 44100, ephemeral = True, autoplay = True)

      print("**************************************************************")
      print("Performance generated successfully")
      print('🎉 Done!')

      #Visual representation of the musical piece
      mm.plot_sequence(sequence)

      #Audio
      #mm.play_sequence(sequence, synth=mm.fluidsynth)

#Example
#performance(4.0, [2,0,1,0,1,1,0,1,0,1,0,1], 30)

Main method to run everything
---

In [ ]:
def mainM():
  import time
  try:
    pieceLength = 20 #seconds
    while True:
      start = time.time()

      # ----------------- IMAGE PROCESSING -----------------
     
      # Brightness.
      image = take_photo()
      #display(Image(image))
      img = cv2.imread(image)
      bright = rgbBright(img)

      # HSV values.
      hsv = photoAnalyse(img)

      # ---------- CREATION OF THE MUSICAL PIECE ----------

      # Notes per second.
      nps = notes_sec(bright)

      # Chromatic scale.
      pitch = pitch_hist(hsv)

      # Generating.
      performance(nps, pitch, pieceLength)

      end = time.time()
      totTime = end - start
      sleepT = pieceLength - totTime
      #print(sleepT)
      if sleepT > 0:
        time.sleep(sleepT)
      
  except KeyboardInterrupt:
    print("Music session concluded")

Main:

In [ ]:
mainM()